# Audio Analysis with SoundLab

This notebook demonstrates SoundLab's comprehensive audio analysis capabilities.

**What you'll learn:**
- Detect tempo and BPM
- Identify musical key
- Measure loudness (LUFS)
- Extract spectral features
- Detect onsets and transients
- Comprehensive audio analysis
- Best practices for analysis

## Setup

Import necessary modules and configure the environment.

In [ ]:
import soundlab
from soundlab.analysis import (
    analyze_audio,
    detect_tempo,
    detect_key,
    measure_loudness,
)
from soundlab.io import load_audio
from pathlib import Path

# For visualization and audio playback
from IPython.display import Audio, display
import matplotlib.pyplot as plt
import numpy as np

print(f"SoundLab version: {soundlab.__version__}")

## 1. Comprehensive Audio Analysis

The simplest way to analyze audio is to use the `analyze_audio()` function, which performs all analyses at once:

In [ ]:
# Path to your audio file
input_file = "../../tests/fixtures/audio/music_like_5s.wav"

# Listen to the input
print("Input audio:")
display(Audio(input_file))

# Perform comprehensive analysis
print("\nAnalyzing audio...")
analysis = analyze_audio(input_file)

# Display summary
print("\nAnalysis Summary:")
print("=" * 50)
for key, value in analysis.summary.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

### Detailed Analysis Results

Let's examine each component of the analysis in detail:

In [ ]:
print("Detailed Analysis Results\n")
print("=" * 50)

# Basic audio properties
print("\n📊 Audio Properties:")
print(f"  Duration: {analysis.duration_seconds:.2f} seconds")
print(f"  Sample rate: {analysis.sample_rate} Hz")
print(f"  Channels: {analysis.channels}")

# Tempo analysis
if analysis.tempo:
    print("\n🎵 Tempo:")
    print(f"  BPM: {analysis.tempo.bpm:.1f}")
    print(f"  Confidence: {analysis.tempo.confidence:.2%}")
    print(f"  Beat interval: {analysis.tempo.beat_interval:.3f}s")
    print(f"  Detected beats: {analysis.tempo.beat_count}")

# Key detection
if analysis.key:
    print("\n🎹 Key:")
    print(f"  Key: {analysis.key.name}")
    print(f"  Confidence: {analysis.key.confidence:.2%}")
    print(f"  Camelot: {analysis.key.camelot}")
    print(f"  Open Key: {analysis.key.open_key}")

# Loudness analysis
if analysis.loudness:
    print("\n🔊 Loudness:")
    print(f"  Integrated LUFS: {analysis.loudness.integrated_lufs:.1f}")
    if analysis.loudness.loudness_range:
        print(f"  Loudness range: {analysis.loudness.loudness_range:.1f} LU")
    if analysis.loudness.true_peak_db:
        print(f"  True peak: {analysis.loudness.true_peak_db:.1f} dBTP")
    print(f"  Broadcast safe: {'✓ Yes' if analysis.loudness.is_broadcast_safe else '✗ No'}")
    print(f"  Streaming optimized: {'✓ Yes' if analysis.loudness.is_streaming_optimized else '✗ No'}")

# Spectral analysis
if analysis.spectral:
    print("\n🌈 Spectral:")
    print(f"  Centroid: {analysis.spectral.spectral_centroid:.1f} Hz")
    print(f"  Bandwidth: {analysis.spectral.spectral_bandwidth:.1f} Hz")
    print(f"  Rolloff: {analysis.spectral.spectral_rolloff:.1f} Hz")
    print(f"  Flatness: {analysis.spectral.spectral_flatness:.3f}")
    print(f"  Brightness: {analysis.spectral.brightness}")
    print(f"  Zero crossing rate: {analysis.spectral.zero_crossing_rate:.4f}")

# Onset detection
if analysis.onsets:
    print("\n⚡ Onsets:")
    print(f"  Onset count: {analysis.onsets.onset_count}")
    print(f"  Average interval: {analysis.onsets.average_interval:.3f}s")
    if analysis.onsets.onset_count > 0:
        print(f"  First onset: {analysis.onsets.onset_times[0]:.3f}s")
        print(f"  Last onset: {analysis.onsets.onset_times[-1]:.3f}s")

## 2. Tempo Detection (BPM)

Detect the tempo and beats in audio:

In [ ]:
# Detect tempo
tempo_result = detect_tempo(input_file)

print("Tempo Detection Results:\n")
print(f"BPM: {tempo_result.bpm:.2f}")
print(f"Confidence: {tempo_result.confidence:.1%}")
print(f"Beat interval: {tempo_result.beat_interval:.3f} seconds")
print(f"Detected beats: {tempo_result.beat_count}")

# Display beat timestamps
if tempo_result.beats:
    print(f"\nFirst 10 beats (seconds):")
    for i, beat_time in enumerate(tempo_result.beats[:10], 1):
        print(f"  Beat {i:2d}: {beat_time:.3f}s")
    
    if tempo_result.beat_count > 10:
        print(f"  ... and {tempo_result.beat_count - 10} more beats")

### Visualizing Beat Detection

In [ ]:
def plot_beats(audio_file, tempo_result, figsize=(14, 4)):
    """
    Visualize detected beats on waveform.
    """
    # Load audio
    audio, sr = load_audio(audio_file)
    
    # Create time axis
    time = np.arange(len(audio)) / sr
    
    # Plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot waveform
    ax.plot(time, audio, alpha=0.6, linewidth=0.5, color='steelblue')
    
    # Plot beats
    for beat_time in tempo_result.beats:
        ax.axvline(x=beat_time, color='red', alpha=0.7, 
                  linewidth=1.5, linestyle='--')
    
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Amplitude', fontsize=12)
    ax.set_title(f'Beat Detection (BPM: {tempo_result.bpm:.1f})', 
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualize beats
if tempo_result.beat_count > 0:
    plot_beats(input_file, tempo_result)

## 3. Key Detection

Detect the musical key of audio:

In [ ]:
# Detect key
key_result = detect_key(input_file)

print("Key Detection Results:\n")
print(f"Key: {key_result.name}")
print(f"Confidence: {key_result.confidence:.1%}")
print(f"Camelot notation: {key_result.camelot}")
print(f"Open Key notation: {key_result.open_key}")

# Show all key correlations
if key_result.all_correlations:
    print("\nAll Key Correlations (top 5):")
    sorted_keys = sorted(key_result.all_correlations.items(), 
                        key=lambda x: x[1], reverse=True)
    for key_name, correlation in sorted_keys[:5]:
        bar = '█' * int(correlation * 20)
        print(f"  {key_name:12s} {correlation:.3f} {bar}")

### Understanding Key Notations

SoundLab provides multiple key notation systems for DJs and musicians:

In [ ]:
print("Key Notation Systems:\n")
print(f"Standard: {key_result.key.value} {key_result.mode.value}")
print(f"Camelot: {key_result.camelot} (for harmonic mixing)")
print(f"Open Key: {key_result.open_key} (alternative DJ notation)")

print("\nCompatible Keys for Mixing:")
# Extract camelot number and letter
camelot_num = int(''.join(filter(str.isdigit, key_result.camelot)))
camelot_letter = key_result.camelot[-1]

# Compatible keys in Camelot system
compatible = [
    f"{camelot_num}{camelot_letter} (same key)",
    f"{camelot_num}{'B' if camelot_letter == 'A' else 'A'} (relative major/minor)",
    f"{camelot_num + 1 if camelot_num < 12 else 1}{camelot_letter} (+1)",
    f"{camelot_num - 1 if camelot_num > 1 else 12}{camelot_letter} (-1)",
]

for comp in compatible:
    print(f"  • {comp}")

## 4. Loudness Analysis (LUFS)

Measure loudness using broadcast standards:

In [ ]:
# Measure loudness
loudness_result = measure_loudness(input_file)

print("Loudness Analysis Results:\n")
print(f"Integrated LUFS: {loudness_result.integrated_lufs:.2f}")

if loudness_result.loudness_range:
    print(f"Loudness Range: {loudness_result.loudness_range:.2f} LU")
    
if loudness_result.true_peak_db:
    print(f"True Peak: {loudness_result.true_peak_db:.2f} dBTP")
    
if loudness_result.dynamic_range_db:
    print(f"Dynamic Range: {loudness_result.dynamic_range_db:.2f} dB")

# Standards compliance
print("\nStandards Compliance:")
print(f"  Broadcast safe (-24 to -14 LUFS): "
      f"{'✓ Yes' if loudness_result.is_broadcast_safe else '✗ No'}")
print(f"  Streaming optimized (-14 LUFS): "
      f"{'✓ Yes' if loudness_result.is_streaming_optimized else '✗ No'}")

### Understanding LUFS Values

Here's a guide to typical LUFS values:

In [ ]:
def interpret_lufs(lufs_value):
    """
    Interpret LUFS value and provide recommendations.
    """
    print(f"Current LUFS: {lufs_value:.1f}\n")
    
    # Reference values
    references = [
        (-6, "Very loud, highly compressed (modern pop/EDM)"),
        (-9, "Loud, compressed (rock, electronic)"),
        (-11, "Moderately loud (hip-hop, R&B)"),
        (-14, "Streaming optimal (Spotify, Apple Music target)"),
        (-16, "Broadcast standard (TV, radio)"),
        (-20, "Quiet/dynamic (acoustic, classical)"),
        (-23, "Very quiet (dialogue, ambient)"),
    ]
    
    print("Reference Levels:")
    for ref_lufs, description in references:
        marker = "← YOU ARE HERE" if abs(lufs_value - ref_lufs) < 2 else ""
        print(f"  {ref_lufs:3d} LUFS: {description} {marker}")
    
    # Recommendations
    print("\nRecommendations:")
    if lufs_value > -10:
        print("  ⚠ Very loud - may sound distorted or fatiguing")
        print("  → Consider reducing gain or limiting")
    elif -14 <= lufs_value <= -10:
        print("  ✓ Good for streaming platforms")
        print("  → Optimal for Spotify, Apple Music, YouTube")
    elif -20 <= lufs_value < -14:
        print("  ✓ Good dynamic range")
        print("  → May be normalized (increased) by streaming services")
    else:
        print("  ⚠ Very quiet")
        print("  → Consider increasing gain to improve audibility")

interpret_lufs(loudness_result.integrated_lufs)

## 5. Spectral Analysis

Extract spectral features to understand frequency content:

In [ ]:
# Get spectral analysis from comprehensive analysis
if analysis.spectral:
    spectral = analysis.spectral
    
    print("Spectral Analysis Results:\n")
    print(f"Spectral Centroid: {spectral.spectral_centroid:.1f} Hz")
    print(f"  (average frequency, weighted by amplitude)")
    print(f"\nSpectral Bandwidth: {spectral.spectral_bandwidth:.1f} Hz")
    print(f"  (spread of frequencies around centroid)")
    print(f"\nSpectral Rolloff: {spectral.spectral_rolloff:.1f} Hz")
    print(f"  (frequency below which 95% of energy is concentrated)")
    print(f"\nSpectral Flatness: {spectral.spectral_flatness:.3f}")
    print(f"  (0 = tonal, 1 = noise-like)")
    print(f"\nZero Crossing Rate: {spectral.zero_crossing_rate:.4f}")
    print(f"  (rate of sign changes, correlates with noisiness)")
    print(f"\nBrightness: {spectral.brightness}")
    print(f"  (qualitative assessment based on centroid)")

### Visualizing Spectral Features

In [ ]:
def plot_spectrum(audio_file, figsize=(14, 8)):
    """
    Plot spectrum and spectral features.
    """
    import librosa
    import librosa.display
    
    # Load audio
    audio, sr = load_audio(audio_file)
    
    # Compute spectrogram
    D = librosa.stft(audio)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    
    # Compute spectral features over time
    centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)
    
    # Plot spectrogram
    img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz', 
                                   ax=ax1, cmap='viridis')
    ax1.set_ylabel('Frequency (Hz)', fontsize=10)
    ax1.set_title('Spectrogram with Spectral Features', 
                 fontsize=12, fontweight='bold')
    
    # Plot spectral centroid and rolloff
    times = librosa.times_like(centroid, sr=sr)
    ax1.plot(times, centroid, label='Centroid', color='red', linewidth=2)
    ax1.plot(times, rolloff, label='Rolloff', color='yellow', linewidth=2)
    ax1.legend(loc='upper right')
    
    fig.colorbar(img, ax=ax1, format='%+2.0f dB')
    
    # Plot waveform
    time = np.arange(len(audio)) / sr
    ax2.plot(time, audio, linewidth=0.5, alpha=0.7, color='steelblue')
    ax2.set_xlabel('Time (seconds)', fontsize=10)
    ax2.set_ylabel('Amplitude', fontsize=10)
    ax2.set_title('Waveform', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualize spectrum
plot_spectrum(input_file)

## 6. Onset Detection

Detect transients and note onsets:

In [ ]:
# Get onset analysis from comprehensive analysis
if analysis.onsets:
    onsets = analysis.onsets
    
    print("Onset Detection Results:\n")
    print(f"Total onsets detected: {onsets.onset_count}")
    print(f"Average interval: {onsets.average_interval:.3f} seconds")
    
    if onsets.onset_count > 0:
        # Calculate onset density
        duration = analysis.duration_seconds
        density = onsets.onset_count / duration
        print(f"Onset density: {density:.1f} onsets/second")
        
        # Display first 10 onsets
        print(f"\nFirst 10 onsets (with strength):")
        for i in range(min(10, onsets.onset_count)):
            time = onsets.onset_times[i]
            strength = onsets.onset_strengths[i]
            bar = '█' * int(strength * 20)
            print(f"  {i+1:2d}. {time:6.3f}s  {strength:.3f} {bar}")
        
        if onsets.onset_count > 10:
            print(f"  ... and {onsets.onset_count - 10} more onsets")

### Visualizing Onsets

In [ ]:
def plot_onsets(audio_file, onset_result, figsize=(14, 6)):
    """
    Visualize detected onsets on waveform.
    """
    # Load audio
    audio, sr = load_audio(audio_file)
    time = np.arange(len(audio)) / sr
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                    gridspec_kw={'height_ratios': [2, 1]})
    
    # Plot waveform with onset markers
    ax1.plot(time, audio, alpha=0.6, linewidth=0.5, color='steelblue')
    
    # Mark onsets
    for onset_time in onset_result.onset_times:
        ax1.axvline(x=onset_time, color='red', alpha=0.7, 
                   linewidth=1.5, linestyle='--')
    
    ax1.set_ylabel('Amplitude', fontsize=10)
    ax1.set_title(f'Onset Detection ({onset_result.onset_count} onsets)', 
                 fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Plot onset strength function
    if onset_result.onset_strengths:
        ax2.plot(onset_result.onset_times, onset_result.onset_strengths, 
                'o-', color='red', markersize=4)
        ax2.set_ylabel('Onset Strength', fontsize=10)
        ax2.set_xlabel('Time (seconds)', fontsize=10)
        ax2.set_title('Onset Strength Function', fontsize=12)
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualize onsets
if analysis.onsets and analysis.onsets.onset_count > 0:
    plot_onsets(input_file, analysis.onsets)

## 7. Batch Analysis

Analyze multiple audio files and compare results:

In [ ]:
def batch_analyze(input_dir, output_csv=None):
    """
    Analyze all audio files in a directory.
    
    Args:
        input_dir: Directory containing audio files
        output_csv: Optional path to save results as CSV
    """
    input_path = Path(input_dir)
    
    # Find all audio files
    audio_extensions = ['.wav', '.mp3', '.flac', '.ogg', '.m4a']
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(input_path.glob(f'*{ext}'))
    
    print(f"Found {len(audio_files)} audio files\n")
    
    results = []
    for i, audio_file in enumerate(audio_files, 1):
        print(f"[{i}/{len(audio_files)}] Analyzing: {audio_file.name}")
        
        try:
            analysis = analyze_audio(str(audio_file))
            
            # Extract key metrics
            result = {
                'filename': audio_file.name,
                'duration': analysis.duration_seconds,
                'sample_rate': analysis.sample_rate,
                'channels': analysis.channels,
            }
            
            if analysis.tempo:
                result['bpm'] = analysis.tempo.bpm
                result['tempo_confidence'] = analysis.tempo.confidence
            
            if analysis.key:
                result['key'] = analysis.key.name
                result['camelot'] = analysis.key.camelot
                result['key_confidence'] = analysis.key.confidence
            
            if analysis.loudness:
                result['lufs'] = analysis.loudness.integrated_lufs
                result['loudness_range'] = analysis.loudness.loudness_range
            
            if analysis.spectral:
                result['brightness'] = analysis.spectral.brightness
                result['spectral_centroid'] = analysis.spectral.spectral_centroid
            
            if analysis.onsets:
                result['onset_count'] = analysis.onsets.onset_count
            
            results.append(result)
            print(f"  ✓ Complete\n")
            
        except Exception as e:
            print(f"  ✗ Error: {e}\n")
            continue
    
    # Display summary table
    if results:
        print("\nAnalysis Summary:")
        print("=" * 80)
        
        # Print header
        print(f"{'File':<30} {'BPM':<8} {'Key':<12} {'LUFS':<8} {'Brightness':<12}")
        print("-" * 80)
        
        # Print results
        for result in results:
            filename = result['filename'][:28]
            bpm = f"{result.get('bpm', 0):.1f}" if 'bpm' in result else 'N/A'
            key = result.get('key', 'N/A')
            lufs = f"{result.get('lufs', 0):.1f}" if 'lufs' in result else 'N/A'
            brightness = result.get('brightness', 'N/A')
            
            print(f"{filename:<30} {bpm:<8} {key:<12} {lufs:<8} {brightness:<12}")
    
    # Save to CSV if requested
    if output_csv and results:
        import csv
        
        with open(output_csv, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=results[0].keys())
            writer.writeheader()
            writer.writerows(results)
        
        print(f"\nResults saved to: {output_csv}")
    
    return results

# Example usage (uncomment to run):
# results = batch_analyze(
#     input_dir="../../tests/fixtures/audio",
#     output_csv="./output/analysis_results.csv"
# )

print("Batch analysis function defined!")

## Best Practices and Tips

### 1. Tempo Detection

- **Works best**: Clear, steady rhythms with prominent drums
- **Challenging**: Rubato, tempo changes, ambient music
- **Tip**: For electronic music, results are usually very accurate
- **Double tempo**: If BPM seems 2x too high/low, it's detecting on/off-beat

### 2. Key Detection

- **Works best**: Tonal music with clear harmonic content
- **Challenging**: Atonal music, chromatic passages, modal music
- **Tip**: Check confidence score - above 0.7 is usually reliable
- **Major/Minor**: Algorithm may confuse relative major/minor keys

### 3. Loudness (LUFS)

**Target LUFS by platform:**
- Spotify: -14 LUFS
- Apple Music: -16 LUFS
- YouTube: -13 LUFS
- Broadcast TV: -23 LUFS (EBU R128)
- Broadcast Radio: -16 LUFS

**Tips:**
- True peak should be below -1.0 dBTP to avoid clipping
- Higher loudness range (LU) = more dynamic
- Streaming services normalize audio to their target

### 4. Spectral Analysis

**Spectral Centroid:**
- < 1500 Hz: Dark, bass-heavy
- 1500-3000 Hz: Balanced
- > 3000 Hz: Bright, treble-heavy

**Spectral Flatness:**
- Close to 0: Tonal (pitched instruments)
- Close to 1: Noise-like (percussion, white noise)

**Zero Crossing Rate:**
- Higher values = more high-frequency content
- Useful for speech/music discrimination

### 5. Onset Detection

- **Use cases**: Rhythmic analysis, beat tracking, audio segmentation
- **High onset density**: Complex, busy music
- **Low onset density**: Sustained, ambient music
- **Tip**: Useful for automatic slicing and sample extraction

### 6. Performance Optimization

- Use `analyze_audio()` for all-in-one analysis (more efficient)
- For specific features, call individual functions
- Enable GPU acceleration where available
- Process shorter segments for very long files

## Summary

In this notebook, you learned how to:

✓ Perform comprehensive audio analysis  
✓ Detect tempo and visualize beats  
✓ Identify musical keys with DJ notations  
✓ Measure loudness for different platforms  
✓ Extract and interpret spectral features  
✓ Detect onsets and transients  
✓ Batch analyze multiple files  
✓ Apply best practices for each analysis type  

**Next Steps:**
- Combine analysis with stem separation
- Use tempo detection for beat-synced effects
- Apply effects processing in notebook 04